<a href="https://colab.research.google.com/github/koya-jp/AA-google-colab-kohya/blob/master/Diffusers_S2D2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Diffusers ライブラリを用いて、画像を生成するスクリプト。**

[Github](https://github.com/koya-jp/AA-google-colab-kohya/blob/master/Diffusers_S2D2.ipynb)

In [1]:
#@title Driveに接続, ライブラリの追加 { display-mode: "form" }

# Driveに接続
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# ライブラリの追加
# !pip install diffusers==0.12.1
# diffusers[torch] 以外の のインストール
!pip install --upgrade diffusers==0.17.1 transformers accelerate scipy ftfy safetensors txt2img >/dev/null 2>&1
!pip install k-diffusion >/dev/null 2>&1


Mounted at /content/drive


In [2]:
#@title 自動切断されないようにするコード { display-mode: "form" }
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

<IPython.core.display.Javascript object>

In [3]:
#@title 「S2D2」のリポジトリを複製し、起動するのに必要な要件をインストール { display-mode: "form" }

%cd /content
!git clone https://github.com/keisuke-okb/S2D2 >/dev/null 2>&1

%cd ./S2D2
!git pull
!touch __init__.py
!pip install -r requirements.txt >/dev/null 2>&1


/content
/content/S2D2
Already up to date.


In [4]:
#@title S2D2の準備、embeddingsを読み込む関数「load_embeddings」の定義 { display-mode: "form" }

import s2d2
import os
import random
import datetime
import torch
from PIL import Image

import diffusers
from diffusers import (StableDiffusionPipeline,StableDiffusionImg2ImgPipeline)
from diffusers.utils import numpy_to_pil
from diffusers.pipelines.stable_diffusion import StableDiffusionPipelineOutput

class StableDiffusionImageGeneratorMod(s2d2.StableDiffusionImageGenerator):
  def __init__(self, sd_safetensor_path: str, device: str="cuda", dtype: torch.dtype=torch.float16):
    self.device = torch.device(device)
    self.pipe = StableDiffusionPipeline.from_pretrained(
      sd_safetensor_path,
      torch_dtype=dtype,
      custom_pipeline="lpw_stable_diffusion"
    ).to(device)
    self.pipe_i2i = StableDiffusionImg2ImgPipeline.from_pretrained(
      sd_safetensor_path,
      torch_dtype=dtype,
      custom_pipeline="lpw_stable_diffusion"
    ).to(device)
    self.pipe.safety_checker = None
    self.pipe_i2i.safety_checker = None
    return

  def load_embeddings(self, safetensors_path: str, fileName: str, token: str):
    if token != "":
      self.pipe.load_textual_inversion(safetensors_path, weight_name=fileName, token=token)
      self.pipe_i2i.load_textual_inversion(safetensors_path, weight_name=fileName, token=token)
    else:
      self.pipe.load_textual_inversion(safetensors_path, weight_name=fileName)
      self.pipe_i2i.load_textual_inversion(safetensors_path, weight_name=fileName)
    return

  %cd /content/

/content


In [10]:
# @title Model、LoRA の読み込み　 { display-mode: "form" }

#モデル
# @markdown *Model ------------------------**
model_path = "/content/drive/MyDrive/StableDiffusion/Model/chilledremix" #@param {type:"string"}

generator = StableDiffusionImageGeneratorMod(
  model_path,
  device="cuda",
)

# @markdown **LoRA1 ------------------------**
LoRA_USE = False #@param {type:"boolean"}
if LoRA_USE == True:
  LoRA="/content/drive/MyDrive/StableDiffusion/Lora/flat2.safetensors" #@param {type:"string"}
  LoRA_alpha = -1.0 #@param {type:"number"}
  generator.load_lora(LoRA, alpha=LoRA_alpha)

# @markdown **LoRA2 ------------------------**
LoRA_USE_2= True #@param {type:"boolean"}
if LoRA_USE_2== True:
  LoRA_2="/content/drive/MyDrive/StableDiffusion/Lora/pretty-cat-rum-sama.safetensors" #@param {type:"string"}
  LoRA_alpha_2 = 0.6 #@param {type:"number"}
  generator.load_lora(LoRA_2, alpha=LoRA_alpha_2)

# @markdown **LoRA3 ------------------------**
LoRA_USE_3= False #@param {type:"boolean"}
if LoRA_USE_3== True:
  LoRA_3=None #@param (string)
  LoRA_alpha_3 = 0.3 #@param {type:"number"}
  pipe = load_safetensors_lora(pipe, LoRA_3, alpha=LoRA_alpha_3)

# @markdown **LoRA4 ------------------------**
LoRA_USE_4= False #@param {type:"boolean"}
if LoRA_USE_4== True:
  LoRA_4="/content/drive/MyDrive/Lora/EkunePOVFellatioV2.safetensors" #@param (string)
  LoRA_alpha_4 = 0.5 #@param {type:"number"}
  pipe = load_safetensors_lora(pipe, LoRA_4, alpha=LoRA_alpha_4)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:649: FutureWarning: 'cached_download' is the legacy way to download files from the HF hub, please consider upgrading to 'hf_hub_download'
  warnings.warn(


In [6]:
# @title Embeddings を設定。

#@markdown **Embeddings 1**
embeddings_USE = False #@param {type:"boolean"}
if embeddings_USE == True:
  safetensors_path = "/content/drive/MyDrive/StableDiffusion/embeddings/EasyNegative.safetensors" #@param {type:"string"}
  file_name = "EasyNegative.safetensors" #@param {type:"string"}
  #@markdown ***トリガーワード（EasyNegativeなど）***
  token = "EasyNegative" #@param {type:"string"}
  generator.load_embeddings(safetensors_path, file_name, token)

#@markdown **Embeddings 2**
embeddings_USE_2 = False #@param {type:"boolean"}
if embeddings_USE_2 == True:
  safetensors_path_2 = "/content/drive/MyDrive/StableDiffusion/embeddings/ulzzang-6500-v1.1.pt" #@param {type:"string"}
  file_name_2 = "ulzzang-6500-v1.1.pt" #@param {type:"string"}
  token_2 = "ulzzang-6500-v1.1" #@param {type:"string"}
  generator.load_embeddings(safetensors_path_2, file_name_2, token_2)

#@markdown **Embeddings 3**
embeddings_USE = True #@param {type:"boolean"}
if embeddings_USE == True:
  safetensors_path_3 = "/content/drive/MyDrive/StableDiffusion/embeddings/pureerosface_v1.pt" #@param {type:"string"}
  file_name_3 = "pureerosface_v1.pt" #@param {type:"string"}
  token_3 = "pureerosface_v1" #@param {type:"string"}
  generator.load_embeddings(safetensors_path_3, file_name_3, token_3)

In [11]:
# @title 画像を生成  （content/drive/MyDrive/StableDiffusion/output/txt2img-〇〇 へ保存）

# 必要なモジュールやライブラリをインポートする
import os
import shutil
from google.colab import files
from PIL.PngImagePlugin import PngInfo

# ファイル名に使う日付と時刻のフォーマットを定義する
file_format = "%Y%m%d_%H%M%S"
i=0

# 現在の日本時間を取得
jst_now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
#txt2img出力画像の保存先
#@@markdown 出力画像を保存するフォルダ → 再表示は＠を一つ消す。
save_path = f"/content/drive/MyDrive/StableDiffusion/output/txt2img-{jst_now.strftime(file_format)}"
os.makedirs(save_path, exist_ok=True)


#@markdown ポジティブプロンプト
prompt = "beautyfull of white fur cat, realistic, (masterpiece:1.3), (extremely detailed CG unity 8k wallpaper), (best quality), (ultra-detailed), (best illustration), (illustration), (sharp focus), Brightness 0.8, Contrast 1.2, detailed, intricate, soft lighting, colorful, colorized, extremely delicate and beautiful, perfect anatomy, perfect hands, beautiful detailed face, beautiful detailed fingers, looking at viewer, no humans, animal focus, white fur, gray fur, whiskers, black eyes, colored sclera, traditional media, " #@param {type:"string"}
#@markdown ネガティブプロンプト
negative = "EasyNegative,(malformed hands, missing fingers, extra digit, fewer digits, bad hands, poorly drawn hands, poorly drawn fingers, missing fingers:1.8),  (face only,body only,legs only,tail only:1.3), have two heads:1.5, lowres, cropped, (worst quality, low quality:1.4), normal quality, jpeg artifacts, desaturated, greyscale, (panel layout, multiple views, multiple angle, two shot, split view, grid view, out of focus, blurry, bokeh:2.0). (text, error, signature, watermark, username, artist name, logo, name, censored:1.5), fat, ugly, mutation, mutated, deformed, bad anatomy, missing arms, three arms, four arms, poorly drawn face, deformed iris, simple iris, heterochromia, long neck, long body, disfigured, extra limb, missing limb, floating limbs, disconnected limbs, (zombie, sketch, interlocked fingers, comic), nsfw" #@param {type:"string"}
#@markdown スケジューラ(サンプラー)
scheduler="dpm++_2m_karras" #@param ["unipc","euler_a","euler","ddim","ddpm","deis","dpm2","dpm++_2s","dpm++_2m","dpm++_2m_karras","dpm++_sde","dpm++_sde_karras","heun","heun_karras","lms","lms_karras","pndm","dpm++_2m_karras"]
#@markdown ステップ数
steps = 20 #@param {type:"integer"}
#@markdown 画像サイズ
img_width = "512" # @param [512, 768, 1024, 1536]
img_height = "512" # @param [512, 768, 1024, 1536]
#@markdown 出力枚数
batch_count = 3 #@param {type: "integer"}
#@markdown CFG
CFG = 7 #@param {type: "number"}
#@markdown シード（-1の時はランダム）
seed = -1 #@param {type: "integer"}
if seed is None or seed == -1:
  inputSeed = random.randint(0, 2147483647)
else:
  valueSeed = seed

#@markdown Hires.fix の有効化
hires_fix = True #@param {type: "boolean"}
enhance_steps = 2 if hires_fix else 1
#@markdown 解像度倍率(乗算後、最も近い8の倍数のサイズとなる)
upscaling_ratio = 2 #@param {type: "number"}
#@markdown アップスケールステップ数
up_steps = 20 #@param {type: "integer"}
#@markdown denoising strength（小さいほど元画像を尊重） （0.5～0.7）が推奨
denoising_strength = 0.60 #@param {type: "number"}


for i in range(batch_count):
  if seed is None or seed == -1:valueSeed = inputSeed + i
  image = generator.diffusion_enhance(
    prompt,
    negative,
    scheduler_name=scheduler,
    num_inference_steps=steps,
    num_inference_steps_enhance=up_steps,
    guidance_scale=CFG,
    width=img_width,
    height=img_height,
    seed=valueSeed,
    upscale_target="latent",
    interpolate_mode="bicubic",
    antialias=True,
    upscale_by=upscaling_ratio,
    enhance_steps=enhance_steps,
    denoising_strength=denoising_strength,
    output_type="pil",
    decode_factor=0.15,
    decode_factor_final=0.18215
  )

  # 現在の日本時間を取得
  jst_now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
  #出力する画像の名前を生成する
  file_name = (jst_now.strftime(file_format)+ "_" + str(valueSeed))
  image_name = file_name + f".png"

  #画像を保存する
  save_location = os.path.join(save_path, image_name)
  #@markdown **メタデータの書き込み**
  save_metadata = True #@param {type: "boolean"}
  if save_metadata:
    metadata = PngInfo()
    metadata.add_text("prompt",(prompt))
    metadata.add_text("negative",(negative))
    metadata.add_text("scheduler",(scheduler))
    metadata.add_text("steps",(str(steps)))
    metadata.add_text("CFG",(str(CFG)))
    metadata.add_text("width",(str(img_width)))
    metadata.add_text("height",(str(img_height)))
    metadata.add_text("seed",str((valueSeed)))
  if hires_fix:
    metadata.add_text("upscaling ratio",str((upscaling_ratio)))
    metadata.add_text("up steps",str((up_steps)))
    metadata.add_text("denoising strength",str((denoising_strength)))
    image.save(save_location, pnginfo=metadata)

  else:
    image.save(save_location)


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
# @title 画像のメタデータを出力

import sys
from PIL import Image
from PIL.PngImagePlugin import PngInfo

#@markdown **保存した画像のパス**
file_path = "/content/drive/MyDrive/StableDiffusion/txt2img_output-20230708_013952/20230708_014048_825314091.png" #@param {type: "string"}
img = Image.open(file_path)

print("Prompt: ",img.text["prompt"])
print("Negative Prompt: ",img.text["negative"])
print("Scheduler: ", img.text["scheduler"])
print("Steps: ",img.text["steps"])
print("CFG: ",img.text["CFG"])
print("Width: ",img.text["width"])
print("Height: ",img.text["height"])
print("Seed: ",img.text["seed"])

try:
  print("Upscaling ratio; ",img.text["upscaling ratio"])
  print("Up steps: ",img.text["up steps"])
  print("Denoising strength: ",img.text["denoising strength"])
except:
  print("Hires.fix was OFF.")

# %%writefile output.txt